# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>
- TAs: Tong Zeng <tozeng@syr.edu>, Priya Matnani <psmatnan@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any of the answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contain `raise NotImplementedError`. This is main to make the `assert` statement fail.
- The tests shown in some cells (i.e., `assert` statements) might not be the only ones used by the professor. Think about cases where your code should run even if it passess all hidden cases.
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

# Part 3: MapReduce with Spark

In [56]:
# Run this code to create the Spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('map-reduce').getOrCreate()
sc = spark.sparkContext

In [57]:
# this RDD will be used throughout this part of the homework
gpa_rdd = sc.parallelize([
 ['2015', 'Fall', 'IST101', 1, 'A', 4],
 ['2015', 'Fall', 'IST195', 3, 'A', 4],
 ['2015', 'Fall', 'IST233', 3, 'B+', 3.3],
 ['2015', 'Fall', 'SOC101', 3, 'A-', 3.7],
 ['2015', 'Fall', 'MAT221', 3, 'C', 2],
 ['2016', 'Fall', 'IST346', 3, 'A', 4],
 ['2016', 'Fall', 'CHE111', 4, 'A-', 3.7],
 ['2016', 'Fall', 'PSY120', 3, 'B+', 3.3],
 ['2016', 'Fall', 'IST256', 3, 'A', 4],
 ['2016', 'Fall', 'ENG121', 3, 'B+', 3.3],
 ['2016', 'Spring', 'GEO110', 3, 'B+', 3.3],
 ['2016', 'Spring', 'MAT222', 3, 'A', 4],
 ['2016', 'Spring', 'SOC121', 3, 'C+', 2.3],
 ['2016', 'Spring', 'BIO240', 3, 'B-', 2.7]])

## Question 3.1 (10 pts) Cumulative GPA with MapReduce (25 pts)

The cumulative GPA reported in any transcript is the average of the numerical score of a course weighted by the credits of such course. Construct a MapReduce job that takes the `gpa_rdd` RDD and returns the cumulative GPA *per season*. 

Each record in `gpa_rdd` contains:
- The year
- The season
- The course code
- The number of credits
- The letter grade
- The number grade

**Hint:** In class, we discussed the MapReduce job for computing avereage. In this case, the key-value pairs will be similar but instead of counting the number of elements considered in the avarage so far, we need to count the credits. Clearly, the key needs to be the season.

In [58]:
def map_weighted_gpa(record):
    # YOUR CODE HERE
    maped_rdd = [record[1],[record[5],record[3]]]
    return maped_rdd
    #raise NotImplementedError()

def reduce_weighted_gpa(value1, value2):
    # YOUR CODE HERE    
    n = value1[0]*value1[1]+value2[0]*value2[1]
    d = value1[1]+value2[1]
    r = [n/d,d]
    return r
    #raise NotImplementedError()

The map job should produce as key the season and value a tuple or list with the grade and credit.

For example, the first element of the map of `gpa_rdd` should be

```python
gpa_rdd.\
    map(map_weighted_gpa).\
    first()
```

```python
['Fall', [4, 1]]
```

In [59]:
gpa_rdd.\
    map(map_weighted_gpa).\
    first()

['Fall', [4, 1]]

In [61]:
##### first result
assert gpa_rdd.\
    map(map_weighted_gpa).\
    first() == ['Fall', [4, 1]]
# the key should be a string
assert gpa_rdd.map(map_weighted_gpa).map(lambda x: type(x[0])).distinct().count() == 1
assert gpa_rdd.map(map_weighted_gpa).map(lambda x: type(x[0])).distinct().first() == str
# all values should be of length 2
assert gpa_rdd.map(map_weighted_gpa).map(lambda x: len(x[1])).distinct().count() == 1
assert gpa_rdd.map(map_weighted_gpa).map(lambda x: len(x[1])).distinct().first() == 2

In [63]:
# there should be two results in the map reduce because there are two semesters
gpa_rdd.\
    map(map_weighted_gpa).\
    reduceByKey(reduce_weighted_gpa).count()

2

In [64]:
assert (gpa_rdd.\
    map(map_weighted_gpa).\
    reduceByKey(reduce_weighted_gpa).collect() == \
    [('Spring', [3.0749999999999997, 12]), 
     ('Fall', [3.503448275862069, 29])]) or \
    (gpa_rdd.\
    map(map_weighted_gpa).\
    reduceByKey(reduce_weighted_gpa).collect() == \
    [('Fall', [3.503448275862069, 29]),
     ('Spring', [3.0749999999999997, 12])])